# Implementação do Simplex  
> SME5901 - Otimização Linear I

---

**Nome:** José Eduardo Saroba Bieco  
**Número USP:** 15856890  

---


# Métodos de Solução Implementados

O núcleo do solver oferece duas abordagens clássicas para a resolução de problemas de PL:

- **Simplex Tabular:** Implementa o algoritmo Simplex tradicional, que mantém e realiza operações de pivoteamento em um tableau completo a cada iteração.
- **Simplex Revisado:** Uma implementação mais eficiente em termos de memória que evita a manutenção do tableau completo. Em vez disso, calcula os componentes necessários a cada passo, como o vetor multiplicador simplex e os custos reduzidos, utilizando a inversa da matriz da base (B_inv).

## Tratamento Automático do Problema

O sistema foi projetado para não exigir que o usuário forneça o problema na forma padrão, realizando as conversões necessárias automaticamente:

- **Método de Duas Fases:** O solver implementa o método de duas fases para lidar com problemas que não possuem uma solução básica factível trivial.
  - **Fase 1:** Quando restrições do tipo ≥ ou = estão presentes, o sistema introduz variáveis artificiais e resolve um problema auxiliar para encontrar uma base factível inicial. Se o valor ótimo da Fase 1 for diferente de zero, o problema é classificado como infactível.
  - **Fase 2:** Partindo da base factível encontrada, o algoritmo procede para encontrar a solução ótima do problema original.
- **Conversão para Forma Padrão:** Antes de iniciar a resolução, o problema é automaticamente convertido para a forma padrão \((Ax = b, x \geq 0)\). Isso inclui a adição de variáveis de folga e excesso para transformar inequações em equações e a garantia de que os termos do lado direito \((b)\) sejam não-negativos.

## Análise e Pré-processamento do Modelo (Parser)

Uma funcionalidade-chave é o parser, que lê modelos de PL a partir de arquivos de texto (.txt) e os prepara para o solver.

- **Flexibilidade de Sintaxe:** O parser interpreta problemas com objetivo de max ou min e restrições com sinais de ≤, ≥, ou =.
- **Tratamento Avançado de Variáveis:** O sistema suporta diferentes domínios para as variáveis de decisão:
  - **Não-negativas:** O padrão, para variáveis \(x_i \geq 0\).
  - **Livres (free):** Variáveis que podem assumir qualquer valor real, transformadas automaticamente em \(x_i = x_i' - x_i''\).
  - **Negativas (negative):** Variáveis com domínio \(x_i \leq 0\), transformadas pela substituição \(x_i = -x_i'\).

## Detecção de Status e Casos Especiais

O solver é capaz de identificar e relatar diversos status e características da solução final:

- **Solução Ótima:** Encontra o valor ótimo e a solução correspondente.
- **Problema Infactível (infeasible):** Detectado ao final da Fase 1, indicando que não há solução que satisfaça todas as restrições.
- **Solução Ilimitada (unbounded):** Identificado durante o teste da razão, quando uma variável pode aumentar seu valor indefinidamente sem violar as restrições.
- **Múltiplas Soluções Ótimas:** Ocorre se, na solução ótima, uma variável não-básica possui custo reduzido igual a zero.
- **Degeneração:** Identificada quando uma ou mais variáveis básicas na solução ótima têm valor nulo.


In [1]:
import numpy as np
from scipy.linalg import inv
import re # from parser import parse_model_from_txt

### Análise dos Parâmetros de Retorno da Função `parse_model_from_txt`

A função `parse_model_from_txt` retorna um único dicionário que encapsula todo o problema de Programação Linear (PL) em um formato estruturado, pronto para ser processado pelo algoritmo Simplex. Abaixo, detalhamos cada chave deste dicionário.

---

#### chave: `c`
* **Tipo**: `list` de `float`
* **Descrição**: Representa o vetor de custos, ou seja, os coeficientes da função objetivo.
* **Detalhes**:
    * Este vetor já está ajustado para o formato de maximização que o Simplex utiliza. Se o problema original for `min z`, a função o converte para `max -z`, e os coeficientes em `c` serão os do problema original multiplicados por -1.
    * Os coeficientes são mapeados para as variáveis do Simplex, que são todas não-negativas. Por exemplo, se a variável original `x1` for livre (transformada em `x1_p - x1_n`), um termo `5x1` no objetivo resultará em `5` e `-5` nas posições correspondentes de `c` para `x1_p` e `x1_n`.

---

#### chave: `A`
* **Tipo**: `list` de `list` de `float`
* **Descrição**: Representa a matriz `A` de coeficientes das restrições.
* **Detalhes**:
    * Cada lista interna corresponde a uma restrição (linha da matriz).
    * As colunas da matriz correspondem às variáveis do Simplex (ex: `x1_p`, `x1_n`, `x2`, ...) na ordem definida internamente pelo parser.

---

#### chave: `b`
* **Tipo**: `list` de `float`
* **Descrição**: Contém os termos independentes, ou seja, os valores do lado direito (RHS - Right-Hand Side) de cada restrição.
* **Detalhes**:
    * A ordem dos valores em `b` corresponde diretamente à ordem das restrições (linhas) na matriz `A`.

---

#### chave: `signs`
* **Tipo**: `list` de `str`
* **Descrição**: Armazena os sinais de relação (`<=`, `>=`, `=`) para cada uma das restrições.
* **Detalhes**:
    * A ordem dos sinais corresponde à ordem das restrições em `A` e `b`.

---

#### chave: `was_min`
* **Tipo**: `bool`
* **Descrição**: Um indicador booleano que "lembra" se o problema original era de minimização.
* **Detalhes**:
    * `True`: O problema original era de minimização.
    * `False`: O problema original era de maximização.
    * **Finalidade**: O solver Simplex é implementado para sempre maximizar. Esta flag é usada ao final do processo para determinar se o valor ótimo encontrado deve ser multiplicado por -1 para fornecer a resposta correta para o problema de minimização original.

---

#### chave: `interpretation_info`
* **Tipo**: `dict`
* **Descrição**: Este dicionário é fundamental. Ele contém todo o "mapa" necessário para traduzir a solução final, que está em termos de variáveis do Simplex, de volta para os termos das variáveis originais do problema.

##### Sub-chave: `sorted_original_vars`
* **Tipo**: `list` de `str`
* **Descrição**: Uma lista com os nomes das variáveis originais do problema (ex: `['x1', 'x2', 'x3']`), ordenadas para garantir uma apresentação de resultados consistente.

##### Sub-chave: `simplex_var_column_names`
* **Tipo**: `list` de `str`
* **Descrição**: Contém os nomes das variáveis que o Simplex efetivamente utiliza. Por exemplo, se `x1` for uma variável livre, esta lista poderia conter `['x1_p', 'x1_n', 'x2']`.

##### Sub-chave: `simplex_vars_map`
* **Tipo**: `dict`
* **Descrição**: O núcleo da tradução. É um dicionário que mapeia o nome de cada variável **original** para um outro dicionário que detalha sua transformação.
    * **`type` (`str`)**: O tipo da variável original. Pode ser:
        * `'non_negative'`: Para variáveis $x_i \ge 0$.
        * `'free'`: Para variáveis livres.
        * `'negative'`: Para variáveis $x_i \le 0$.
    * **`cols_parser` (`list[int]`)**: Os índices das colunas que a variável original ocupa na matriz `A` e no vetor `c`. Uma variável não-negativa terá um único índice (ex: `[0]`), enquanto uma livre terá dois (ex: `[0, 1]`).
    * **`mult` (`int`)**: Um multiplicador usado na transformação dos coeficientes. É `1` para variáveis não-negativas/livres e `-1` para variáveis negativas (na transformação `x_orig = -x_prime`).
    * **`simplex_names` (`list[str]`)**: Os nomes formais dados às variáveis do Simplex que representam a variável original (ex: `['x1_p', 'x1_n']`).

In [2]:
def parse_model_from_txt(filepath):
    """
    Analisa um arquivo de texto (.txt) contendo um modelo de Programação Linear (PL)
    e o converte para um formato matricial adequado para o solver Simplex.

    A função extrai a função objetivo, as restrições e as especificações de
    domínio das variáveis (padrão >= 0, livre ou negativa). Ela transforma
    automaticamente o modelo para a forma padrão, onde todas as variáveis de
    decisão são não-negativas, e prepara um dicionário com todas as informações
    necessárias para a resolução e posterior interpretação dos resultados.

    Formato do Arquivo de Modelo Esperado:
    - A primeira linha deve ser a função objetivo (ex: "max 3x1 + 5x2").
    - A linha "s.t." (subject to) deve separar o objetivo das restrições.
    - Cada restrição deve estar em uma nova linha (ex: "x1 + 2x2 <= 10").
    - Sinais aceitos para restrições: '<=', '>=', '='.
    - Especificações de domínio são opcionais e podem vir antes ou depois das
      restrições (ex: "x1 free", "x3 negative"). Variáveis não especificadas
      são consideradas não-negativas (>= 0) por padrão.

    Args:
        filepath (str): O caminho para o arquivo .txt contendo o modelo de PL.

    Returns:
        dict: Um dicionário contendo a estrutura do problema de PL processado.
              Este dicionário está pronto para ser passado como argumento para a
              classe Simplex. A estrutura detalhada deste dicionário de retorno
              é explicada na seção "Análise do Dicionário de Retorno".
    """
    with open(filepath, 'r') as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]

    objective_line = lines[0]
    constraints_lines = []
    domain_spec_lines = [] # Para linhas como "x1 free", "x2 negative"

    # Identificar s.t. e separar as linhas
    s_t_found = False
    for line in lines[1:]:
        if 's.t.' in line.lower():
            s_t_found = True
            continue
        if not s_t_found: # Linhas antes de s.t. que não são objetivo (talvez comentários ou domínios globais)
            domain_spec_lines.append(line) # Ou tratar como erro/ignorar
            continue

        # Após s.t.
        if any(op in line for op in ['<=', '>=', '=']):
            constraints_lines.append(line)
        else:
            # Linhas após s.t. que não são restrições são consideradas especificações de domínio
            domain_spec_lines.append(line)

    is_max = 'max' in objective_line.lower()
    is_min = 'min' in objective_line.lower()
    if not (is_max or is_min):
        raise ValueError("A função objetivo deve começar com 'max' ou 'min'")

    # 1. Identificar todas as variáveis originais x_i mencionadas no modelo
    all_text = ' '.join(lines)
    original_var_names_set = set(re.findall(r'x\d+', all_text))
    if not original_var_names_set:
        raise ValueError("Nenhuma variável (ex: x1, x2) encontrada no modelo.")
    sorted_original_vars = sorted(list(original_var_names_set), key=lambda v: int(v[1:]))


    # 2. Processar linhas de especificação de domínio
    free_vars = set()
    negative_vars = set() # Para x_i <= 0

    for line in domain_spec_lines:
        vars_in_line = set(re.findall(r'x\d+', line))
        if not vars_in_line:
            continue
        
        line_lower = line.lower()
        if 'free' in line_lower:
            free_vars.update(vars_in_line)
        elif 'negative' in line_lower: # Palavra-chave para x_i <= 0
            negative_vars.update(vars_in_line)
        # Poderia adicionar "non-negative" ou "positive" se precisasse de declaração explícita,
        # mas o padrão é não-negativo.

    # Checagem de conflitos (ex: uma variável não pode ser free e negative)
    if free_vars.intersection(negative_vars):
        conflicting_vars = free_vars.intersection(negative_vars)
        raise ValueError(f"Variáveis declaradas como 'free' e 'negative' ao mesmo tempo: {conflicting_vars}")

    # 3. Mapear variáveis originais para variáveis do Simplex (todas >= 0)
    #    e determinar seus multiplicadores de coeficiente.
    
    # simplex_vars_map: {'x1': {'type':'non_negative', 'cols_parser': [0], 'mult': 1, 'simplex_names': ['x1']}, ...}
    simplex_vars_map = {}
    simplex_var_column_names = [] # Nomes das colunas no A_transformed, c_transformed
    current_simplex_col = 0

    for var_orig in sorted_original_vars:
        if var_orig in free_vars:
            p_name = f"{var_orig}_p"
            n_name = f"{var_orig}_n"
            simplex_var_column_names.extend([p_name, n_name])
            simplex_vars_map[var_orig] = {
                'type': 'free',
                'cols_parser': [current_simplex_col, current_simplex_col + 1],
                'mult': 1, # Não usado diretamente para 'free', pois os coefs são divididos
                'simplex_names': [p_name, n_name]
            }
            current_simplex_col += 2
        elif var_orig in negative_vars:
            prime_name = f"{var_orig}_prime" # x_orig = -x_prime ONDE x_prime >= 0
            simplex_var_column_names.append(prime_name)
            simplex_vars_map[var_orig] = {
                'type': 'negative',
                'cols_parser': [current_simplex_col],
                'mult': -1, # Coeficientes de x_orig são multiplicados por -1 para x_prime
                'simplex_names': [prime_name]
            }
            current_simplex_col += 1
        else: # Default: não-negativa
            simplex_var_column_names.append(var_orig)
            simplex_vars_map[var_orig] = {
                'type': 'non_negative',
                'cols_parser': [current_simplex_col],
                'mult': 1,
                'simplex_names': [var_orig]
            }
            current_simplex_col += 1
            
    num_simplex_vars = len(simplex_var_column_names)

    # 4. Construir vetor de custos `c_transformed` para as variáveis do Simplex
    c_transformed = np.zeros(num_simplex_vars)
    obj_expr = objective_line.lower().replace('max', '').replace('min', '').strip()
    
    for term_match in re.finditer(r'([+-]?\s*\d*\.?\d*)\s*(x\d+)', obj_expr):
        coeff_str = term_match.group(1).replace(' ', '')
        var_orig = term_match.group(2)

        if coeff_str == '' or coeff_str == '+':
            coeff_val = 1.0
        elif coeff_str == '-':
            coeff_val = -1.0
        else:
            coeff_val = float(coeff_str)

        if var_orig not in simplex_vars_map: continue # Variável no obj não reconhecida

        map_info = simplex_vars_map[var_orig]
        cols = map_info['cols_parser']

        if map_info['type'] == 'free':
            c_transformed[cols[0]] += coeff_val
            c_transformed[cols[1]] -= coeff_val # Para -x_n
        else: # non_negative ou negative
            # Para x_negativo (x = -x'), se o termo é C*x, vira C*(-x') = (-C)*x'
            # O multiplicador já cuida disso: coeff_val * map_info['mult']
            c_transformed[cols[0]] += coeff_val * map_info['mult']
            
    if is_min:
        c_transformed = -c_transformed # Converte min para max -Z

    # 5. Construir matriz de restrições `A_transformed`
    A_transformed = []
    b_transformed = []
    signs_transformed = []

    for constr_line in constraints_lines:
        # Separar LHS, sinal, e RHS
        match = re.match(r'(.+?)\s*(<=|>=|=)\s*([^<>=]+)', constr_line)
        if not match:
            raise ValueError(f"Formato de restrição inválido: {constr_line}")
        
        lhs_expr, sign, rhs_str = match.groups()
        signs_transformed.append(sign)
        b_transformed.append(float(rhs_str.strip()))
        
        row_coeffs = np.zeros(num_simplex_vars)
        for term_match in re.finditer(r'([+-]?\s*\d*\.?\d*)\s*(x\d+)', lhs_expr):
            coeff_str = term_match.group(1).replace(' ', '')
            var_orig = term_match.group(2)

            if coeff_str == '' or coeff_str == '+':
                coeff_val = 1.0
            elif coeff_str == '-':
                coeff_val = -1.0
            else:
                coeff_val = float(coeff_str)

            if var_orig not in simplex_vars_map: continue

            map_info = simplex_vars_map[var_orig]
            cols = map_info['cols_parser']

            if map_info['type'] == 'free':
                row_coeffs[cols[0]] += coeff_val
                row_coeffs[cols[1]] -= coeff_val
            else: # non_negative ou negative
                row_coeffs[cols[0]] += coeff_val * map_info['mult']
        A_transformed.append(list(row_coeffs))

    # Informações para reverter a solução para as variáveis originais
    interpretation_info = {
        'sorted_original_vars': sorted_original_vars,
        'simplex_vars_map': simplex_vars_map, # Contém tipo, colunas e multiplicador
        'simplex_var_column_names': simplex_var_column_names # Nomes das vars do simplex
    }

    return {
        'c': list(c_transformed), # Coeficientes para as variáveis do Simplex
        'A': A_transformed,       # Matriz A para as variáveis do Simplex
        'b': b_transformed,
        'signs': signs_transformed,
        'was_min': is_min,
        'interpretation_info': interpretation_info
    }

# Simplex

In [3]:
class Simplex:
    def __init__(self, c_from_parser, A_from_parser, b, signs, was_min=False, interpretation_info=None):
        """
        Inicializa o solver Simplex com os dados do problema de Programação Linear.

        Args:
            c_from_parser (list): Vetor de custos para as variáveis já processadas pelo parser.
            A_from_parser (list): Matriz de coeficientes das restrições para as variáveis do parser.
            b (list): Vetor de termos independentes (lado direito) das restrições.
            signs (list): Lista com os sinais de cada restrição ('<=', '>=', '=').
            was_min (bool, optional): True se o problema original era de minimização. Default é False.
            interpretation_info (dict, optional): Dicionário com dados para mapear a solução de volta às variáveis originais.
        """

        self.c_parser_vars = np.array(c_from_parser, dtype=float)
        self.A_parser_vars = np.array(A_from_parser, dtype=float)
        self.b_orig = np.array(b, dtype=float)
        self.signs = signs
        self.was_min = was_min
        self.interpretation_info = interpretation_info
        self.m, self.n_parser_vars = self.A_parser_vars.shape

          # Atributos que serão preenchidos durante a preparação do problema
        self.A = None
        self.c = None
        self.b = None
        self.num_vars = None

    def _prepare_problem(self):
        """
            Converte o problema de PL para a Forma Padrão (A'x = b', x >= 0, b' >= 0).

            Esta função realiza as seguintes etapas:
            1. Garante que todos os elementos de 'b' (lado direito) sejam não-negativos,
            multiplicando as restrições por -1 quando necessário.
            2. Adiciona variáveis de folga e excesso para converter as inequalidades
            em igualdades.
            3. Monta a matriz 'A' e o vetor de custos 'c' finais, que incluem
            as variáveis de decisão originais e as de folga/excesso.
        """

        self.b = np.copy(self.b_orig)
        temp_A = np.copy(self.A_parser_vars)
        
        for i in range(self.m):
            if self.b[i] < 0:
                self.b[i] *= -1
                temp_A[i, :] *= -1
                if self.signs[i] == '<=': self.signs[i] = '>='
                elif self.signs[i] == '>=': self.signs[i] = '<='

        # Calcula o número de variáveis de folga/excesso
        num_slack_vars = sum(1 for sign in self.signs if sign != '=')
        self.num_vars = self.n_parser_vars + num_slack_vars
        
        # Cria a matriz A e o vetor c na forma padrão
        self.A = np.zeros((self.m, self.num_vars))
        self.A[:, :self.n_parser_vars] = temp_A
        self.c = np.zeros(self.num_vars)
        self.c[:self.n_parser_vars] = self.c_parser_vars

        # Adiciona as variáveis de folga/excesso
        slack_ptr = self.n_parser_vars
        for i, sign in enumerate(self.signs):
            if sign == '<=':
                self.A[i, slack_ptr] = 1.0
                slack_ptr += 1
            elif sign == '>=':
                self.A[i, slack_ptr] = -1.0
                slack_ptr += 1

    # --------------------------------------------------------------------------
    # FLUXO DE SOLUÇÃO PRINCIPAL
    # --------------------------------------------------------------------------
    def solve(self, method='revised'):
        """
            Ponto de entrada principal para resolver o problema de Programação Linear.

            A função prepara o problema para a forma padrão e, em seguida, seleciona
            o método de solução (Tabular ou Revisado) para encontrar a solução ótima.

            Args:
                method (str, optional): O método a ser usado. Pode ser 'tabular' ou 'revised'.
                                        Default é 'revised'.

            Returns:
                dict: Um dicionário contendo o status final da otimização ('optimal', 
                    'infeasible', 'unbounded', etc.), a solução encontrada (se houver) e 
                    outras informações relevantes.
        """
        self._prepare_problem()
        
        if method == 'revised':
            result = self._solve_revised()
        elif method == 'tabular':
            result = self._solve_tabular()
        else:
            raise ValueError("Método inválido. Escolha 'tabular' ou 'revised'.")

        return self._format_final_solution(result)

    # --------------------------------------------------------------------------
    # LÓGICA PARA O MÉTODO REVISADO
    # --------------------------------------------------------------------------
    def _solve_revised(self):
        """
            Orquestra a solução usando o método Simplex Revisado de duas fases.

            Fase 1: Encontra uma Solução Básica Factível inicial para o problema.
            Fase 2: Usa a base factível da Fase 1 para encontrar a solução ótima.

            Returns:
                dict: Dicionário com o resultado do processo. Se a Fase 1 determinar
                    que o problema é infactível, o status 'infeasible' é retornado.
                    Caso contrário, retorna o resultado da Fase 2.
        """

        # Fase 1: Encontrar uma base factível
        phase1_result = self._run_phase1_revised()
        if phase1_result.get('status') != 'feasible':
            return phase1_result
        
        initial_base_indices = phase1_result['base']
        
         # Fase 2: Encontrar a solução ótima
        return self._revised_simplex_engine(self.A, self.b, self.c, initial_base_indices)

    def _run_phase1_revised(self):
        """
            Executa a Fase 1 do Simplex Revisado para encontrar uma base factível.

            Constrói e resolve um problema auxiliar cujo objetivo é minimizar a soma das
            variáveis artificiais. Se o valor ótimo deste problema for zero, uma
            base factível foi encontrada. Caso contrário, o problema original é infactível.

            Returns:
                dict: Um dicionário com o status.
                    - {'status': 'feasible', 'base': [...]} se uma base factível for encontrada.
                    - {'status': 'infeasible'} se o problema original for infactível.
        """

        # Identifica restrições que precisam de variáveis artificiais
        artificial_rows = {i for i, sign in enumerate(self.signs) if sign in ['>=', '=']}
        if not artificial_rows:
            # Base trivial com vars de folga (se houver)
            slack_indices = [self.n_parser_vars + i for i, s in enumerate(self.signs) if s != '=']
            return {'status': 'feasible', 'base': slack_indices}

        # Constrói o problema da Fase 1
        num_artificial = len(artificial_rows)
        A_phase1 = np.hstack([self.A, np.zeros((self.m, num_artificial))])
        c_phase1 = np.zeros(self.A.shape[1] + num_artificial)
        c_phase1[self.A.shape[1]:] = -1.0
        
        # Adiciona variáveis artificiais e define a base inicial da Fase 1
        initial_base_phase1 = [-1] * self.m
        art_ptr = self.A.shape[1]
        slack_ptr = self.n_parser_vars
        for i in range(self.m):
            if i in artificial_rows:
                A_phase1[i, art_ptr] = 1.0
                initial_base_phase1[i] = art_ptr
                art_ptr += 1
            else: # Restrição <=
                initial_base_phase1[i] = slack_ptr
                slack_ptr += 1
        
        result_phase1 = self._revised_simplex_engine(A_phase1, self.b, c_phase1, initial_base_phase1)

        # Verifica o resultado da Fase 1
        if result_phase1.get('status') != 'optimal' or abs(result_phase1.get('value', 0)) > 1e-9:
            return {'status': 'infeasible'}

        final_base_phase1 = result_phase1['final_basis_indices']
        
        # Verifica se alguma variável artificial permaneceu na base
        if any(b >= self.A.shape[1] for b in final_base_phase1):
             return {'status': 'error_redundant_constraint', 'message': 'Não foi possível expulsar as variáveis artificiais da base. O modelo pode ter restrições redundantes.'}

        return {'status': 'feasible', 'base': final_base_phase1}

    def _revised_simplex_engine(self, A, b, c, initial_basic_indices):
        """
            Motor principal que executa as iterações do algoritmo Simplex Revisado.

            Args:
                A (np.array): A matriz de coeficientes das restrições (forma padrão).
                b (np.array): O vetor do lado direito (forma padrão).
                c (np.array): O vetor de custos (forma padrão).
                initial_basic_indices (list): Lista de índices das variáveis na base inicial.

            Returns:
                dict: Um dicionário descrevendo o resultado da otimização ('optimal', 'unbounded', etc.).
        """
        basic_indices = np.array(initial_basic_indices, dtype=int)
        num_vars = A.shape[1]
        
        for _ in range(self.m * num_vars * 2): # Limite de iterações
            B = A[:, basic_indices]
            try:
                B_inv = inv(B)
            except np.linalg.LinAlgError:
                return {'status': 'error_singular_matrix'}
            
            non_basic_indices = np.setdiff1d(np.arange(num_vars), basic_indices)
            c_b = c[basic_indices]
            x_b = B_inv @ b
            y = c_b @ B_inv
            cj_zj = c[non_basic_indices] - y @ A[:, non_basic_indices]

            # Condição de otimalidade
            if np.all(cj_zj <= 1e-9):
                sol = np.zeros(num_vars)
                sol[basic_indices] = x_b
                return {'status': 'optimal', 'solution': sol, 'value': c_b @ x_b,
                        'is_degenerate': np.any(np.isclose(x_b, 0)),
                        'has_multiple_solutions': np.any(np.isclose(cj_zj, 0)),
                        'final_basis_indices': basic_indices, 'final_B_inv': B_inv}
            
            # Escolha da variável que entra na base
            entering_idx = non_basic_indices[np.argmax(cj_zj)]
            d = B_inv @ A[:, entering_idx]
            
            # Condição de solução ilimitada
            if np.all(d <= 1e-9): return {'status': 'unbounded'}
            
            # Teste da razão para escolher a variável que sai da base
            ratios = np.array([x_b[i] / d[i] if d[i] > 1e-9 else np.inf for i in range(self.m)])
            leaving_row = np.argmin(ratios)
            basic_indices[leaving_row] = entering_idx
        
        return {'status': 'max_iterations_reached'}

    # --------------------------------------------------------------------------
    # LÓGICA PARA O MÉTODO TABULAR
    # --------------------------------------------------------------------------
    def _solve_tabular(self):
        """
            Orquestra a solução usando o método Simplex Tabular de duas fases.

            Fase 1: Constrói um tableau e encontra uma solução básica factível.
            Fase 2: Usa o tableau resultante da Fase 1 para encontrar a solução ótima.

            Returns:
                dict: Dicionário com o resultado do processo. Se a Fase 1 determinar
                    que o problema é infactível, o status 'infeasible' é retornado.
                    Caso contrário, retorna o resultado da Fase 2.
        """

        # Fase 1: Construir e resolver o tableau
        tableau, basic_indices, artificial_indices, status = self._build_and_run_phase1_tabular()
        if status != 'optimal':
            return {'status': status} # Retorna 'infeasible' ou outro erro

        # Fase 2: Preparar e resolver o tableau para a otimização
        tableau, basic_indices = self._prepare_phase2_tableau(tableau, basic_indices, artificial_indices)
        return self._tabular_simplex_engine(tableau, basic_indices, self.c)
        
    def _build_and_run_phase1_tabular(self):
        """
            Cria e resolve o tableau da Fase 1 para encontrar uma base factível.

            Constrói um tableau inicial adicionando variáveis de folga, excesso e artificiais
            conforme necessário. Em seguida, resolve este tableau usando um objetivo auxiliar
            para minimizar as variáveis artificiais.

            Returns:
                tuple: Uma tupla contendo:
                    - tableau (np.array): O tableau final da Fase 1.
                    - basic_indices (list): Os índices da base factível encontrada.
                    - artificial_indices (list): Os índices das variáveis artificiais.
                    - status (str): 'optimal' se factível, 'infeasible' caso contrário.
        """


        num_artificial = sum(1 for sign in self.signs if sign in ['>=', '='])
        num_slack = self.num_vars - self.n_parser_vars
        
        # Monta o tableau da Fase 1
        tableau_width = self.n_parser_vars + num_slack + num_artificial + 1
        tableau = np.zeros((self.m, tableau_width))
        tableau[:, :self.n_parser_vars] = self.A[:, :self.n_parser_vars]
        tableau[:, -1] = self.b
        
        basic_indices = [-1] * self.m
        artificial_indices = []
        
        c_phase1 = np.zeros(tableau_width - 1)
        
        # Preenche o tableau com as variáveis de folga, excesso e artificiais
        slack_ptr = self.n_parser_vars
        art_ptr = self.n_parser_vars + num_slack
        
        for i in range(self.m):
            sign = self.signs[i]
            if sign == '<=':
                tableau[i, slack_ptr] = 1.0
                basic_indices[i] = slack_ptr
                slack_ptr += 1
            else: # >= ou =
                if sign == '>=':
                    tableau[i, slack_ptr] = -1.0
                    slack_ptr += 1
                tableau[i, art_ptr] = 1.0
                basic_indices[i] = art_ptr
                artificial_indices.append(art_ptr)
                c_phase1[art_ptr] = -1.0 # max -w
                art_ptr += 1

        # Resolve o problema da Fase 1
        result = self._tabular_simplex_engine(tableau, basic_indices, c_phase1)
        
        # Verifica se a Fase 1 encontrou uma solução factível
        if result.get('status') != 'optimal' or abs(result.get('value', 0)) > 1e-9:
            return None, None, None, 'infeasible'
        
        return result['tableau'], result['final_basis_indices'], artificial_indices, 'optimal'

    def _prepare_phase2_tableau(self, tableau, basic_indices, artificial_indices):
        """
            Modifica o tableau final da Fase 1 para iniciar a Fase 2.

            A principal tarefa é remover as colunas correspondentes às variáveis
            artificiais e ajustar os índices da base de acordo.

            Args:
                tableau (np.array): O tableau resultante da Fase 1.
                basic_indices (list): A lista de índices da base da Fase 1.
                artificial_indices (list): A lista de índices das variáveis artificiais.

            Returns:
                tuple: Uma tupla contendo o novo tableau e a nova lista de índices da base.
        """
        # Remove colunas das variáveis artificiais
        cols_to_keep = [i for i in range(tableau.shape[1] - 1) if i not in artificial_indices]
        tableau = tableau[:, cols_to_keep + [tableau.shape[1] - 1]]
        
        # Mapeia os índices da base antigos para os novos
        map_old_to_new = {old: new for new, old in enumerate(cols_to_keep)}
        new_basic_indices = [map_old_to_new[b] for b in basic_indices]
        
        return tableau, new_basic_indices

    def _tabular_simplex_engine(self, tableau_init, basic_indices_init, c_original):
        """
            Motor principal que executa as iterações do algoritmo Simplex Tabular.

            Args:
                tableau_init (np.array): O tableau inicial para a fase atual.
                basic_indices_init (list): A lista inicial de índices da base.
                c_original (np.array): O vetor de custos original para esta fase.

            Returns:
                dict: Um dicionário descrevendo o resultado da otimização ('optimal', 'unbounded', etc.).
        """
        tableau = np.copy(tableau_init)
        basic_indices = list(basic_indices_init)
        num_vars = tableau.shape[1] - 1
        
        # Limite de iterações para evitar loops infinitos
        for _ in range(self.m * num_vars * 2):
            # Calcula os custos reduzidos (linha cj - zj)
            cb = c_original[basic_indices]
            zj = cb @ tableau[:, :-1]
            cj_zj = c_original - zj

            # Zera os custos reduzidos das variáveis básicas por precisão numérica
            cj_zj[basic_indices] = 0

           # Condição de otimalidade
            if np.all(cj_zj <= 1e-9):
                solution = np.zeros(num_vars)
                for i, idx in enumerate(basic_indices):
                    solution[idx] = tableau[i, -1]
                
                # Checa por múltiplas soluções nas variáveis NÃO básicas
                non_basic_indices = np.setdiff1d(np.arange(num_vars), basic_indices)
                has_multiple_solutions = np.any(np.isclose(cj_zj[non_basic_indices], 0))

                return {
                    'status': 'optimal',
                    'solution': solution,
                    'value': cb @ solution[basic_indices],
                    'is_degenerate': np.any(np.isclose(tableau[:, -1], 0)),
                    'has_multiple_solutions': has_multiple_solutions,
                    'tableau': tableau,
                    'final_basis_indices': basic_indices
                }

            # Escolhe a variável para entrar na base
            entering_col = np.argmax(cj_zj)
            
            # Condição de solução ilimitada
            if np.all(tableau[:, entering_col] <= 1e-9):
                return {'status': 'unbounded'}

            # Teste da razão para escolher a variável que sai da base
            ratios = np.array([tableau[i, -1] / tableau[i, entering_col] if tableau[i, entering_col] > 1e-9 else np.inf for i in range(self.m)])
            leaving_row = np.argmin(ratios)
            
            # Realiza o pivoteamento para atualizar o tableau
            pivot_element = tableau[leaving_row, entering_col]
            if abs(pivot_element) < 1e-9:
                return {'status': 'error_numerical_instability'}
            
            tableau[leaving_row, :] /= pivot_element
            for i in range(self.m):
                if i != leaving_row:
                    tableau[i, :] -= tableau[i, entering_col] * tableau[leaving_row, :]
            
            # Atualiza a base
            basic_indices[leaving_row] = entering_col

        return {'status': 'max_iterations_reached'}

    # --------------------------------------------------------------------------
    # PÓS-PROCESSAMENTO
    # --------------------------------------------------------------------------
    def _format_final_solution(self, result):
        """
        Formata o dicionário de resultado bruto em uma saída final para o usuário.

        Esta função traduz a solução, que está em termos de variáveis internas do 
        Simplex (incluindo folga, etc.), de volta para as variáveis originais do problema.
        Também ajusta o valor final da função objetivo se o problema era de minimização.

        Args:
            result (dict): O dicionário de resultado bruto dos motores Simplex.

        Returns:
            dict: O dicionário final formatado, contendo a solução em termos das
                  variáveis originais, o valor ótimo e o status.
        """
        # Se o resultado não for ótimo, retorna diretamente
        if result.get('status') != 'optimal':
            return result
        
        if self.interpretation_info == None:
            return result

        sol_vector = result['solution']
        final_sol = {}

        # Usa o 'interpretation_info' para reverter a solução para as variáveis originais
        for var, info in self.interpretation_info['simplex_vars_map'].items():
            cols = info['cols_parser']
            if info['type'] == 'free':
                final_sol[var] = sol_vector[cols[0]] - sol_vector[cols[1]]
            else:
                final_sol[var] = sol_vector[cols[0]] * info['mult']
        
        ordered_sol = [final_sol[v] for v in self.interpretation_info['sorted_original_vars']]
        
        # Recalcula o valor da função objetivo com base nos coeficientes do problema original
        final_value = self.c_parser_vars @ ordered_sol
        
        return {'status': 'optimal', 'solution': ordered_sol,
                'value': -final_value if self.was_min else final_value,
                'is_degenerate': result.get('is_degenerate', False),
                'has_multiple_solutions': result.get('has_multiple_solutions', False)}


# Modelo a modelo

`modelos/slides/aula4_pt1_slide83.txt`

$$
\begin{aligned}
\text{Minimize} \quad & -3x_1 - 2x_2 \\
\text{subject to} \quad
& 0.5x_1 + 0.3x_2 \leq 3 \\
& 0.1x_1 + 0.2x_2 \leq 1 \\
& 0.4x_1 + 0.5x_2 \leq 3 \\
& x_1, x_2 \geq 0
\end{aligned}
$$


In [4]:
model = parse_model_from_txt(r'./modelos/slides/aula4_pt1_slide83.txt')
# print(model)

simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  #interpretation_info=model['interpretation_info']
                  )

# result = simplex.solve("tabular")
# print("tabular")
# print(result)
result = simplex.solve("revised")
print("revised")
print(result)

revised
{'status': 'optimal', 'solution': array([4.61538462, 2.30769231, 0.        , 0.07692308, 0.        ]), 'value': 18.461538461538456, 'is_degenerate': False, 'has_multiple_solutions': False, 'final_basis_indices': array([0, 3, 1]), 'final_B_inv': array([[ 3.84615385, -0.        , -2.30769231],
       [ 0.23076923,  1.        , -0.53846154],
       [-3.07692308,  0.        ,  3.84615385]])}


`modelos/slides/aula4_pt2_slide30.txt`

$$
\begin{aligned}
\text{Minimize} \quad & -x_1 - 2x_2 \\
\text{subject to} \quad
& x_1 + x_2 \leq 6 \\
& -x_1 + x_2 \leq 4 \\
& x_1 \geq 0 \\
& x_2 \geq 0
\end{aligned}
$$


In [5]:
model = parse_model_from_txt(r'./modelos/slides/aula4_pt2_slide30.txt')
# print(model)

simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  #interpretation_info=model['interpretation_info']
                  )

# result = simplex.solve("tabular")
# print("tabular")
# print(result)
result = simplex.solve("revised")
print("revised")
print(result)

revised
{'status': 'optimal', 'solution': array([1., 5., 0., 0., 1., 5.]), 'value': 11.0, 'is_degenerate': False, 'has_multiple_solutions': False, 'final_basis_indices': array([4, 5, 0, 1]), 'final_B_inv': array([[ 0.5, -0.5, -1. , -0. ],
       [ 0.5,  0.5,  0. , -1. ],
       [ 0.5, -0.5,  0. ,  0. ],
       [ 0.5,  0.5,  0. ,  0. ]])}


`modelos/degenerada_1.txt`

$$
\begin{aligned}
\text{Maximize} \quad & 3x_1 + 2x_2 \\
\text{subject to} \quad
& x_1 + x_2 \leq 4 \\
& 2x_1 + x_2 \leq 5 \\
& 2x_1 + 2x_2 \leq 8 \\
& x_1, x_2 \geq 0
\end{aligned}
$$


In [6]:
model = parse_model_from_txt(r'./modelos/degenerada_1.txt')
# print(model)

simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  #interpretation_info=model['interpretation_info']
                  )

# result = simplex.solve("tabular")
# print("tabular")
# print(result)
result = simplex.solve("revised")
print("revised")
print(result)

revised
{'status': 'optimal', 'solution': array([1., 3., 0., 0., 0.]), 'value': 9.0, 'is_degenerate': True, 'has_multiple_solutions': False, 'final_basis_indices': array([1, 0, 4]), 'final_B_inv': array([[ 2., -1.,  0.],
       [-1.,  1.,  0.],
       [-2.,  0.,  1.]])}


`modelos/ilimitada_1.txt`

$$
\begin{aligned}
\text{Maximize} \quad & x_1 + x_2 \\
\text{subject to} \quad
& x_1 - x_2 \leq 1 \\
& -x_1 + x_2 \leq 1 \\
& x_1, x_2 \geq 0
\end{aligned}
$$


In [7]:
model = parse_model_from_txt(r'./modelos/ilimitada_1.txt')
# print(model)

simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  #interpretation_info=model['interpretation_info']
                  )

# result = simplex.solve("tabular")
# print("tabular")
# print(result)
result = simplex.solve("revised")
print("revised")
print(result)

revised
{'status': 'unbounded'}


`modelos/infactivel_1.txt`

$$
\begin{aligned}
\text{Minimize} \quad & 2x_1 + 3x_2 \\
\text{subject to} \quad
& x_1 + x_2 \geq 5 \\
& x_1 \leq 4 \\
& x_1 \geq 5 \\
& x_1, x_2 \geq 0
\end{aligned}
$$

In [8]:
model = parse_model_from_txt(r'./modelos/infactivel_1.txt')
# print(model)

simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  #interpretation_info=model['interpretation_info']
                  )

# result = simplex.solve("tabular")
# print("tabular")
# print(result)
result = simplex.solve("revised")
print("revised")
print(result)

revised
{'status': 'infeasible'}


`modelos/infactivel_2.txt`

$$
\begin{aligned}
\text{Maximize} \quad & x_1 + x_2 \\
\text{subject to} \quad
& x_1 + x_2 \leq 2 \\
& x_1 + x_2 \geq 4 \\
& x_1, x_2 \geq 0
\end{aligned}
$$


In [9]:
model = parse_model_from_txt(r'./modelos/infactivel_2.txt')
# print(model)

simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  #interpretation_info=model['interpretation_info']
                  )

# result = simplex.solve("tabular")
# print("tabular")
# print(result)
result = simplex.solve("revised")
print("revised")
print(result)

revised
{'status': 'infeasible'}


`modelos/modelo_online1.txt`
$$
\begin{aligned}
\text{Maximize} \quad & x_1 + 2x_2 - x_3 \\
\text{subject to} \quad
& 2x_1 + x_2 + x_3 \leq 14 \\
& 4x_1 + 2x_2 + 3x_3 \leq 28 \\
& 2x_1 + 5x_2 + 5x_3 \leq 30 \\
& x_1, x_2, x_3 \geq 0
\end{aligned}
$$


In [10]:
model = parse_model_from_txt(r'./modelos/modelo_online1.txt')
#print(model)

simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  #interpretation_info=model['interpretation_info']
                  )

# result = simplex.solve("tabular")
# print("tabular")
# print(result)
result = simplex.solve("revised")
print("revised")
print(result)

revised
{'status': 'optimal', 'solution': array([5., 4., 0., 0., 0., 0.]), 'value': 13.0, 'is_degenerate': True, 'has_multiple_solutions': False, 'final_basis_indices': array([0, 4, 1]), 'final_B_inv': array([[ 0.625,  0.   , -0.125],
       [-2.   ,  1.   ,  0.   ],
       [-0.25 ,  0.   ,  0.25 ]])}


`modelos/video_pedro_munari.txt`
- Link https://youtu.be/2_5YfX29F2k?si=9yli9shuwdGzzhOQ

$$
\begin{aligned}
\text{Maximize} \quad & x_1 + 2x_2 \\
\text{subject to} \quad
& x_1 + x_2 \leq 6 \\
& x_1 - x_2 \leq 4 \\
& -x_1 + x_2 \leq 4 \\
& x_1, x_2 \geq 0
\end{aligned}
$$


In [11]:
model = parse_model_from_txt(r'./modelos/video_pedro_munari.txt')
# print(model)

simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  #interpretation_info=model['interpretation_info']
                  )

# result = simplex.solve("tabular")
# print("tabular")
# print(result)
result = simplex.solve("revised")
print("revised")
print(result)

revised
{'status': 'optimal', 'solution': array([1., 5., 0., 8., 0.]), 'value': 11.0, 'is_degenerate': False, 'has_multiple_solutions': False, 'final_basis_indices': array([0, 3, 1]), 'final_B_inv': array([[ 0.5, -0. , -0.5],
       [ 0. ,  1. ,  1. ],
       [ 0.5,  0. ,  0.5]])}


In [14]:
model = parse_model_from_txt(r'./modelos/teste.txt')
# print(model)W

simplex = Simplex(model['c'], model['A'], 
                  model['b'], model['signs'], was_min=model['was_min'], 
                  #interpretation_info=model['interpretation_info']
                  )

# result = simplex.solve("tabular")
# print("tabular")
# print(result)
result = simplex.solve("revised")
print("revised")
print(result)

revised
{'status': 'optimal', 'solution': array([4., 0., 2., 0., 0., 0., 2., 2., 5., 3.]), 'value': 14.0, 'is_degenerate': False, 'has_multiple_solutions': False, 'final_basis_indices': array([0, 2, 6, 7, 8, 9]), 'final_B_inv': array([[ 1. , -0.5,  0. , -0. ,  0. , -0. ],
       [ 0. ,  0.5,  0. , -0. ,  0. , -0. ],
       [-1. ,  0.5,  1. , -0. ,  0. , -0. ],
       [ 0. ,  0. ,  0. ,  1. ,  0. , -0. ],
       [ 0. ,  0.5,  0. ,  0. ,  1. , -0. ],
       [ 0. , -0.5,  0. ,  0. ,  0. ,  1. ]])}
